In [1]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
# decorator
from langchain.agents import tool
import requests
import tempfile
from langchain.document_loaders import BSHTMLLoader
from langchain.memory import ConversationBufferMemory

In [2]:
@tool
def get_latest_page(text: str) -> str:
    '''Accesses a URL that is provided as a string, extracts the text content and returns this content as a string as well. 
    '''
    try:
        resp = requests.get(text)

        with tempfile.NamedTemporaryFile(delete=False) as fp:
            fp.write(resp.content)
            fp.close()

            loader = BSHTMLLoader(fp.name)
            data = loader.load() 
            resp = data[0].page_content
    except Exception as e:
        print(e)
        resp = ''
    
    return resp

In [3]:
llm = ChatOpenAI(temperature=0)

In [4]:
tools = load_tools(['llm-math'], llm=llm)

In [5]:
tools += [get_latest_page]

In [6]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [7]:
agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory = memory,
    verbose = True
)

In [9]:
agent.run('Can you extract the contents of this webpage https://techstrong.ai/aiops/the-rise-of-shadow-ai/ and then generate a summary of the extracted text?')



> Entering new AgentExecutor chain...
I need to extract the contents of the webpage and then generate a summary of the text. I can use the get_latest_page tool to access the URL and get the text content.
Action: get_latest_page
Action Input: "https://techstrong.ai/aiops/the-rise-of-shadow-ai/"
Observation: 









The Rise of Shadow AI - Techstrong.ai





























































 




Skip to content








Toggle NavigationLatestArticlesFeaturesNewsVideosRelated SitesTechstrong GroupContainer JournalSecurity BoulevardDigital CxOTechstrong ResearchDevOps ChatDevOps DozenDevOps TVTechstrong TVTechstrong TV PodcastTechstrong TV TwitchAboutMedia Kit 


Search for:










Toggle NavigationAI at the EdgeAI CareersBuilding With AIConversational AICybersecurity AIData EngineeringData ScienceMoreAerospaceAIOpsGenerative AIGovernmentMachine/Deep LearningPrompt EngineeringTransportation/Travel

 

Home » The Rise of Shadow AIThe Rise of Shadow AIPublished

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I have successfully extracted the contents of the webpage.
Action: None`